In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
model = YOLO('yolo11s-cls.pt')

dataset_path = '/content/drive/MyDrive/Hackathon/newdata'
output_dir_on_drive = '/content/drive/MyDrive/Hackathon/train_epoch10_yolo11s_cls_newdata'

results = model.train(
    data=dataset_path,
    epochs=20,
    imgsz=224,
    project=output_dir_on_drive,
    name='Durian_Diseases',
    exist_ok=True
)

print("Hoàn tất huấn luyện!")
print("Kết quả được lưu tại thư mục 'train_epoch10_yolo11s_cls_newdata/Durian_Diseases'")

In [ ]:
## kiểm tra độ chuẩn của model

from ultralytics import YOLO
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
from PIL import Image
import requests
from io import BytesIO


model_path = '/content/drive/MyDrive/Hackathon/train_epoch10_yolo11n_cls/Durian_Diseases/weights/best.pt'
model = YOLO(model_path)


image_source = 'https://media-cdn-v2.laodong.vn/storage/newsportal/2025/3/30/1484187/Sau-Rieng-16.jpg'



results = model.predict(source=image_source)


result = results[0]
class_names = result.names
probabilities = result.probs.data


print("--- Xác Suất Dự Đoán ---")
for i, prob in enumerate(probabilities):
    class_name = class_names[i]
    print(f"{class_name}: {prob.item() * 100:.2f}%")


best_prob_index = probabilities.argmax()
predicted_class_name = class_names[best_prob_index.item()]
confidence_score = probabilities[best_prob_index.item()]

print("\n--- KẾT QUẢ ---")
print(f" Dự đoán: {predicted_class_name}")
print(f" Độ tin cậy: {confidence_score * 100:.2f}%")


print("\n--- Hình Ảnh Dự Đoán ---")

img_array = result.orig_img

img_bgr = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)


text = f"Predicted: {predicted_class_name} ({confidence_score * 100:.2f}%)"
(text_width, text_height), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)
cv2.rectangle(img_bgr, (5, 5), (10 + text_width, 35 + text_height), (0,0,0), -1)

# Vẽ chữ kết quả lên ảnh
cv2.putText(
    img_bgr,
    text,
    (10, 30),
    cv2.FONT_HERSHEY_SIMPLEX, # Font chữ
    0.9, # Kích thước chữ
    (36, 255, 12), # Màu chữ (xanh lá cây)
    2 # Độ dày chữ
)

# Hiển thị ảnh trong output của Colab
cv2_imshow(img_bgr)